In [ ]:
##灰度处理  高斯模糊去噪声 canny边缘去噪   扫描线填充法   获取动态区域   蒙版图像    得到霍夫变换的图像    外推线作为预测    重叠区域叠加
#阴影干扰下检测效果不错的

In [40]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import math

In [25]:
#读取图片
def read(path):
  return mpimg.imread(path)
#显示图片
def show(image):
  print('This image is:', type(image), 'with dimensions:', image.shape)
  plt.imshow(image)
#保存图片
def save(image, path = 'test.png'):
  plt.imsave(path, image)
#转换成彩色图片
def convert_to_color(image):
  return np.dstack((image, image, image))

In [26]:
#转换为灰色图片
def gray(raw_image):
  '''convert the colored imaged to grayscale image'''
  return cv2.cvtColor(raw_image, cv2.COLOR_RGB2GRAY)  
#高斯模糊去噪声
def reduce_noise(gray_image, kernel_size):
  '''reduce noise of grayscale image using gausian blur'''
  return cv2.GaussianBlur(gray_image, (kernel_size, kernel_size), 0)

In [27]:
#canny边缘去噪
def get_edges(blur_image, low_threshold, high_threshold):
  '''find edges using canny transform algorithm'''
  return cv2.Canny(blur_image, low_threshold, high_threshold)

In [28]:
#扫描线填充法
def get_vertices(x, y, axc, bxc, cyc, dyc, maxyc = 1.0, maxxc = 1.0, startxc = 0.0):
  ax = int(axc*x)
  bx = int(bxc*x)
  cy = int(cyc*y)
  dy = int(dyc*y)
  maxy = int(maxyc*y)
  maxx = int(maxxc*x)
  startx = int(startxc*x)
  bottom_left = (startx, maxy)
  top_left = (ax, cy)
  top_right = (bx, dy)
  bottom_right = (maxx, maxy)
  vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
  return vertices 

In [29]:
#获取动态感兴趣区域
def get_roi(edge_image, vertices, ignore_value = 255):
  '''get the polygon to be used to block out everything in the image except the region of interest'''
  roi = np.zeros_like(edge_image)
  cv2.fillPoly(roi, vertices, ignore_value)
  return roi

#将二值图片的效果反转既黑色变白色，白色变黑色
def mask(edge_image, roi):
  '''block out everything in the image except the edges in the region of interest'''
  return cv2.bitwise_and(edge_image, roi)

In [30]:
#累计渐进霍夫变换
def get_lines(masked_edge_image, rho, theta_coef, min_votes, min_line_length, max_line_gap):
  '''convert edges into lines using hough transform algorithm '''
  theta = theta_coef*np.pi/180
  return cv2.HoughLinesP(masked_edge_image, rho, theta, min_votes, np.array([]), minLineLength = min_line_length, maxLineGap = max_line_gap)


In [31]:
#把霍夫变换后得到的线段画到一副图片上
def draw(lines, image, color=[255, 0, 0], thickness = 2, thresh = 0.5):
  ''' draw the lines on a blank image'''
  lined_image = np.copy(image)*0
  
  if lines is not None:
    for line in lines:
      for x1,y1,x2,y2 in line:
        slope, intercept = np.polyfit((x1,x2), (y1,y2), 1)
        if abs(slope) > thresh:
          cv2.line(lined_image, (x1, y1), (x2, y2), color, thickness)
        
  return lined_image

In [9]:
# equation of a line: y = slope*x + intercept 
# left lane has a positive slope, right lane has a negative slope 
#外推线  其实是预测的线
def extrapolate_lines(lines, image, color=[255, 0, 0], thickness = 10, 
                      positive_thresh = 0.5, negative_thresh = 0.5):
    
  imshape = image.shape
  image = np.copy(image)*0
    
  #initialize minimum and maximum y coordinate 
  minimum_y = image.shape[0] 
  maximum_y = image.shape[0]
  
  #initialize groups of values into empty lists
  left_slopes = []
  left_xs = []
  left_ys = []   
  right_slopes = []
  right_xs = []
  right_ys = []
  
  # segregate the small line segments into the left lane group or right lane group
  if lines is not None:  
    for line in lines:
      for x1,y1,x2,y2 in line:
        
        # get the slope and intercept of the line (as defined by two points) using the polyfit function
        slope, intercept = np.polyfit((x1,x2), (y1,y2), 1)
            
        if (slope > positive_thresh): #if positive slope, put value to left lane group
          left_slopes += [slope]
          left_xs += [x1, x2]
          left_ys += [y1, y2]
        elif (slope < negative_thresh): #if negative slope, put value to right lane group
          right_slopes += [slope]
          right_xs += [x1, x2]
          right_ys += [y1, y2]
        
        # update the minimum y_coordinate based on values seen
        minimum_y = min(min(y1, y2), minimum_y)
  
  #average all the values in each group to get the slope, x, and y
  left_slope = np.mean(left_slopes)
  left_x = np.mean(left_xs)
  left_y = np.mean(left_ys)
  right_slope = np.mean(right_slopes)
  right_x = np.mean(right_xs)
  right_y = np.mean(right_ys)
    
  #derive the intercept using the equation of the line and average value
  left_intercept = left_y - (left_slope * left_x)
  right_intercept = right_y - (right_slope * right_x)

  if ((len(left_slopes) > 0) and (len(right_slopes) > 0)): #make sure we have points in each group
    #derive the x coordinate using the equation of the lines and derived values
    upper_left_x = int((minimum_y - left_intercept) / left_slope)
    lower_left_x = int((maximum_y - left_intercept) / left_slope)
    upper_right_x = int((minimum_y - right_intercept) / right_slope)
    lower_right_x = int((maximum_y - right_intercept) / right_slope)
    
    #draw the line based on two points 
    cv2.line(image, (upper_left_x, minimum_y), (lower_left_x, maximum_y), color, thickness)
    cv2.line(image, (upper_right_x, minimum_y), (lower_right_x, maximum_y), color, thickness)
    
  return image

In [32]:
#将两个图像叠加
def overlap(first_image, second_image, α = 0.8, β = 0.5, λ = 0.0):
    '''first_image * α + second_image * β + λ (colored)'''
    return cv2.addWeighted(first_image, α, second_image, β, λ)

In [33]:
#算法的参数
param = {
   #blur parameters
  'kernel_size': 5, 
    
   #canny transform parameters
  'canny_lo': 100, 
  'canny_hi': 200, 

  #region of interest parameters
  'ax_coef': 10.0/25,
  'bx_coef': 14.0/25,
  'cy_coef': 0.6,
  'dy_coef': 0.6,
  'maxy_coef': 1.0,
  'maxx_coef': 1.0,
  'startx_coef': 0.0,

  #hough parameters
  'rho': 1, 
  'theta_coef': 1, 
  'min_votes': 30, 
  'min_line_length': 20, 
  'max_line_gap': 20
}

In [38]:
#管道
def pipeline(path, file, name, param = param):
    
  name = path + name

  raw_image = read(path + file)
  image = read(path + file)
  save(image, path = name + '1-raw.png')
  
  gray_image = gray(image)
  save(gray_image, path = name + '2-grey.png')
    
  blur_image = reduce_noise(gray_image, param['kernel_size'])
  save(blur_image, path = name + '3-blur.png')
    
  edge_image = get_edges(blur_image, param['canny_lo'], param['canny_hi'])
  save(edge_image, path = name + '4-edges.png')
  
  x = image.shape[1]
  y = image.shape[0]
  vertices = get_vertices(x, y, param['ax_coef'], param['bx_coef'],
                                param['cy_coef'], param['dy_coef'],
                                param['maxy_coef'], param['maxx_coef'], param['startx_coef'])
  
  roi = get_roi(edge_image, vertices)
  save(roi, path = name + '5-roi.png')
  
  mask_image = mask(edge_image, roi)
  save(mask_image, path = name + '6-mask.png')
  
  lines = get_lines(mask_image, param['rho'], param['theta_coef'],
                           param['min_votes'], param['min_line_length'],
                           param['max_line_gap'])
   
  line_image = draw(lines, raw_image)
  save(line_image, path = name + '7-lines.png')
  
  
  line_image2 = extrapolate_lines(lines, raw_image)
  save(line_image2, path = name + '8-lines.png')

  image = overlap(line_image, raw_image)
  save(image, path = name + '9-overlap.png')

  image = overlap(line_image2, raw_image)
  save(image, path = name + '10-overlap.png')
   

pipeline(path = 'test_images/test/', file = 'solidWhiteCurve.jpg', name = 'A')
pipeline(path = 'test_images/test/', file = 'solidWhiteRight.jpg', name = 'B')
pipeline(path = 'test_images/test/', file = 'solidYellowCurve.jpg', name = 'C')
pipeline(path = 'test_images/test/', file = 'solidYellowCurve2.jpg', name = 'D')
pipeline(path = 'test_images/test/', file = 'solidYellowLeft.jpg', name = 'E')
pipeline(path = 'test_images/test/', file = 'whiteCarLaneSwitch.jpg', name = 'F')



In [14]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [15]:

def process_image(image):

  raw_image = np.copy(image)
  gray_image = gray(image)    
  blur_image = reduce_noise(gray_image, param['kernel_size'])
    
  edge_image = get_edges(blur_image, param['canny_lo'], param['canny_hi'])
  
  x, y = image.shape[1], image.shape[0]
  vertices = get_vertices(x, y, param['ax_coef'], param['bx_coef'],
                                param['cy_coef'], param['dy_coef'],
                                param['maxx_coef'], param['maxy_coef'],
                                param['startx_coef'])
  roi = get_roi(edge_image, vertices)  
  mask_image = mask(edge_image, roi)
  
  lines = get_lines(mask_image, param['rho'], param['theta_coef'],
                           param['min_votes'], param['min_line_length'],
                           param['max_line_gap'])
   
  #line_image = draw(lines, raw_image)  
  #result = overlap(line_image, raw_image)
 
  line_image2 = extrapolate_lines(lines, raw_image) 
  result = overlap(line_image2, raw_image)
  return result

In [17]:
white_output = 'white.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████▊| 221/222 [00:03<00:00, 68.00it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 3.66 s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [19]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████▉| 681/682 [00:10<00:00, 67.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 10.5 s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [21]:
p = {
   #blur parameters
  'kernel_size': 3, 
    
   #canny transform parameters
  'canny_lo': 50, 
  'canny_hi': 150, 

  #region of interest parameters
  'ax_coef': 0.41,
  'bx_coef': 0.60,
  'cy_coef': 0.65,
  'dy_coef': 0.65,
  'maxy_coef': 0.9,
  'maxx_coef': 0.85,
  'startx_coef': 0.15,

  #hough parameters
  'rho': 1, 
  'theta_coef': 1, 
  'min_votes': 20, 
  'min_line_length': 50, 
  'max_line_gap': 20
}

def process(image):

  raw_image = np.copy(image)

  gray_image = gray(image)    
  blur_image = reduce_noise(gray_image, p['kernel_size'])
    
  edge_image = get_edges(blur_image, p['canny_lo'], p['canny_hi'])
  
  #return convert_to_color(edge_image) #check if canny parameters detect edges of lanes

  x = image.shape[1]
  y = image.shape[0]
  vertices = get_vertices(x, y, p['ax_coef'], p['bx_coef'],
                                p['cy_coef'], p['dy_coef'],
                                p['maxy_coef'], p['maxx_coef'], p['startx_coef'])
  
  roi = get_roi(edge_image, vertices)
  mask_image = mask(edge_image, roi)
  
  #return overlap(convert_to_color(roi), raw_image) #check if roi is good
    
  lines = get_lines(mask_image, p['rho'], p['theta_coef'],
                                p['min_votes'], p['min_line_length'],
                                p['max_line_gap'], )
   
  #line_image = draw(lines, raw_image, thresh = 0.5)  
  #result = overlap(line_image, raw_image)
 
  line_image2 = extrapolate_lines(lines, raw_image, 
                                  positive_thresh = 0.5, negative_thresh = -0.5) 
  result = overlap(line_image2, raw_image)
  
  return result

In [22]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████| 251/251 [00:07<00:00, 32.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 8.56 s


In [23]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [42]:
import newspaper

In [43]:
newspaper.languages()


Your available languages are:

input code		full name
  ar			  Arabic
  da			  Danish
  de			  German
  el			  Greek
  en			  English
  es			  Spanish
  fa			  Persian
  fi			  Finnish
  fr			  French
  he			  Hebrew
  hi			  Hindi
  hu			  Hungarian
  id			  Indonesian
  it			  Italian
  ko			  Korean
  mk			  Macedonian
  nb			  Norwegian (Bokmål)
  nl			  Dutch
  no			  Norwegian
  pl			  Polish
  pt			  Portuguese
  ru			  Russian
  sv			  Swedish
  sw			  Swahili
  tr			  Turkish
  uk			  Ukrainian
  vi			  Vietnamese
  zh			  Chinese



In [44]:
cnn_paper = newspaper.build('http://cnn.com')
for article in cnn_paper.articles:
    print(article.url)

http://cnn.com/2017/03/04/vr/how-to-watch-vr
http://cnn.com/2018/03/10/us/california-veterans-home-shooting/index.html
http://money.cnn.com/2018/03/05/news/economy/china-belt-road-debt-pakistan-laos/index.html
http://cnn.com/2018/03/04/asia/chinese-military-budget-intl/index.html
http://cnn.com/2018/03/04/asia/chinas-npc-xi-jinping-four-things-intl/index.html
http://cnn.com/2018/03/03/politics/trump-maralago-remarks/index.html
http://cnn.com/2018/03/07/health/lesotho-abortions-asequals-intl/index.html
http://cnn.com/2018/03/07/africa/amina-mohammed-gender-parity-asequals-iwd-intl/index.html
http://cnn.com/2018/03/09/africa/cape-town-day-zero-crisis-intl/index.html
http://cnn.com/2018/03/09/africa/kenya-odinga-kenyatta-joint-speech/index.html
http://cnn.com/2018/03/08/politics/global-gag-early-health-impact/index.html
http://cnn.com/2017/10/03/americas/us-gun-statistics/index.html
http://cnn.com/2018/03/08/americas/colombia-farc-leader-presidential-bid/index.html
http://cnn.com/2018/03/

http://cnn.com/2018/03/02/opinions/hard-schools-arent-the-answer-to-shootings-shmukler-opinion/index.html
http://cnn.com/2018/03/04/opinions/parkland-columbine-tactics-opinion-gagliano/index.html
http://cnn.com/2018/03/03/opinions/slavery-lessons-in-us-schools-opinion-love/index.html
http://cnn.com/2018/03/02/opinions/francisco-cantu-immigration-border-jane-carr-opinion/index.html
http://cnn.com/2018/03/01/opinions/hollywood-disability-new-normal-opinion-novic/index.html
http://cnn.com/2018/02/28/opinions/raising-boys-violence-opinion-drexler/index.html
http://cnn.com/2017/05/19/opinions/fdr-knew-about-ringling-bros-circus-lauder/index.html
http://cnn.com/2017/04/15/opinions/overheating-of-great-barrier-reef-doubilet/index.html
http://cnn.com/2017/07/25/opinions/nudity-hollywood-opinion-davis/index.html
http://cnn.com/2017/03/19/opinions/oxford-comma-ambiguity-opinion-mcwhorter/index.html
http://cnn.com/2017/04/02/opinions/stress-killing-us-keating-opinion/index.html
http://cnn.com/201

http://cnn.com/travel/article/tohoku-natural-attractions/index.html
http://cnn.com/travel/article/japan-tohoku-best-sake/index.html
http://money.cnn.com/news/made-in-india/
http://money.cnn.com/2017/12/01/technology/india-electric-cars-mahindra-2030/index.html
http://money.cnn.com/2017/11/10/technology/flipkart-smartphone-billion-capture-india/index.html
http://money.cnn.com/2017/11/30/news/economy/india-economy-gdp-growth-recovery/index.html
http://cnn.com/TRANSCRIPTS/2018.03.01.html
http://cnn.com/TRANSCRIPTS/2018.03.02.html
http://cnn.com/TRANSCRIPTS/2018.03.03.html
http://cnn.com/TRANSCRIPTS/2018.03.04.html
http://cnn.com/TRANSCRIPTS/2018.03.05.html
http://cnn.com/TRANSCRIPTS/2018.03.06.html
http://cnn.com/TRANSCRIPTS/2018.03.07.html
http://cnn.com/TRANSCRIPTS/2018.03.08.html
http://cnn.com/TRANSCRIPTS/2018.03.09.html
http://cnn.com/TRANSCRIPTS/2018.03.10.html
http://cnn.com/2018/03/10/politics/trump-pennsylvania-speech-rick-saccone/index.html
http://cnn.com/2018/03/10/politics/job

http://cnn.com/2017/10/09/sport/gallery/what-a-shot-sports-1010/index.html
http://cnn.com/2017/10/02/sport/gallery/what-a-shot-sports-1003/index.html
http://cnn.com/2017/09/25/sport/gallery/what-a-shot-sports-0925/index.html
http://cnn.com/2018/02/25/sport/pyeongchang-2018-winter-olympics-top-moments/index.html
http://cnn.com/2018/03/01/sport/rodchenkov-jim-walden-ioc-olympic-russia-roc/index.html
http://cnn.com/2018/03/10/us/california-veterans-home-shooting-victims/index.html
http://cnn.com/2018/03/10/us/pomona-cops-shot/index.html
http://cnn.com/2018/03/09/us/winter-storm-noreaster-weather/index.html
http://cnn.com/2018/03/10/us/oprah-on-parkland-cnntv/index.html
http://cnn.com/2018/03/09/politics/pentagon-memo-trump-military-parade/index.html
http://cnn.com/2018/03/09/politics/stormy-daniels-sarah-sanders-briefing/index.html
http://cnn.com/2018/03/09/politics/north-korea-trump-obama-bush-clinton/index.html
http://cnn.com/2018/03/09/us/florida-gov-scott-gun-bill/index.html
http://mo

http://cnn.com/2018/02/26/opinions/president-xi-term-limits-opinion-intl/index.html
http://cnn.com/2018/02/09/world/patrick-ho-corruption-china-africa/index.html
http://cnn.com/2018/02/04/health/china-left-behind-kids-photography-intl/index.html
http://money.cnn.com/2018/01/25/news/economy/dandong-china-north-korea-sanctions/index.html
http://cnn.com/2017/12/25/health/china-dog-cloning-for-disease-intl/index.html
http://cnn.com/2017/12/08/asia/china-migrant-workers-evictions-beijing/index.html
http://cnn.com/2017/11/23/asia/china-lawyers-disappeared/index.html
http://cnn.com/travel/article/china-lunar-new-year-journey-home-motorbike-intl/index.html
http://cnn.com/travel/article/expansion-hong-kong-disneyland/index.html
http://cnn.com/travel/article/what-to-do-shenzhen-china/index.html
http://money.cnn.com/2018/02/26/investing/daimler-geely-china-electric-cars/index.html
http://money.cnn.com/2018/02/21/news/economy/china-us-soybeans-trade-trump/index.html
http://money.cnn.com/2018/02/23

http://cnnpressroom.blogs.cnn.com/2015/12/
http://cnnpressroom.blogs.cnn.com/2015/11/
http://cnnpressroom.blogs.cnn.com/2015/10/
http://cnnpressroom.blogs.cnn.com/2015/09/
http://cnnpressroom.blogs.cnn.com/2015/08/
http://cnnpressroom.blogs.cnn.com/2015/07/
http://cnnpressroom.blogs.cnn.com/2015/06/
http://cnnpressroom.blogs.cnn.com/2015/05/
http://cnnpressroom.blogs.cnn.com/2015/04/
http://cnnpressroom.blogs.cnn.com/2015/03/
http://cnnpressroom.blogs.cnn.com/2015/02/
http://cnnpressroom.blogs.cnn.com/2015/01/
http://cnnpressroom.blogs.cnn.com/2014/12/
http://cnnpressroom.blogs.cnn.com/2014/11/
http://cnnpressroom.blogs.cnn.com/2014/10/
http://cnnpressroom.blogs.cnn.com/2014/09/
http://cnnpressroom.blogs.cnn.com/2014/08/
http://cnnpressroom.blogs.cnn.com/2014/07/
http://cnnpressroom.blogs.cnn.com/2014/06/
http://cnnpressroom.blogs.cnn.com/2014/05/
http://cnnpressroom.blogs.cnn.com/2014/04/
http://cnnpressroom.blogs.cnn.com/2014/03/
http://cnnpressroom.blogs.cnn.com/2014/02/
http://cnnp

http://cnn.com/2018/03/08/health/house-committee-hearing-on-flu-bn/index.html
http://cnn.com/2018/03/07/health/new-brain-cells-adulthood-study/index.html
http://cnn.com/2018/03/08/health/parkland-trauma-israel/index.html
http://www.cnn.com/interactive/2018/02/health/parallels/sex/
http://inhealth.cnn.com/advances-in-immunotherapy/how-immunotherapy-treats-advanced-renal-cell-carcinoma?did=t1_rss7
http://cnn.com/2018/03/07/health/brain-food-drayer/index.html
http://cnn.com/2017/12/29/health/flu-recovery-tips/index.html
http://cnn.com/2018/01/31/health/flu-symptoms/index.html
http://cnn.com/2018/01/24/health/how-flu-kills/index.html
http://cnn.com/2018/01/15/health/honey-health-benefits/index.html
http://cnn.com/2017/12/11/health/diet-coke-trump-health-effects/index.html
http://cnn.com/2017/12/06/health/protein-powder-pros-cons/index.html
http://cnn.com/2017/12/08/health/avocados-health-benefits-food-as-fuel-explainer/index.html
http://cnn.com/2018/02/07/health/weight-loss-medication-dray

http://cnn.com/2018/02/27/opinions/voice-from-eastern-ghouta-opinion-intl/index.html
http://cnn.com/2018/03/08/middleeast/hajj-sexual-abuse-asequals-iwd-intl/index.html
http://cnn.com/2018/03/06/middleeast/iran-women-protest-hijab-online-asequals-intl/index.html
http://cnn.com/2018/02/28/middleeast/mars-oman-mission-oewf/index.html
http://cnn.com/2018/01/12/middleeast/saudi-women-attend-first-soccer-match/index.html
http://cnn.com/style/article/everyday-middle-east/index.html
http://cnn.com/style/article/mashpi-lodge-ecuador/index.html
http://cnn.com/style/article/hairaat-fine-jewellery/index.html
http://cnn.com/style/article/in-between-maysaloun-hamoud/index.html
http://money.cnn.com/news/marketplace-middle-east/index.html
http://money.cnn.com/2017/10/05/news/economy/saudi-king-in-russia/index.html
http://cnn.com/style/article/louvre-abu-dhabi-museum-opening/index.html
http://cnn.com/2016/12/14/architecture/tower-dubai-creek-tallest-skyscraper/index.html
http://cnn.com/2018/02/16/vr/c

In [45]:
for category in cnn_paper.category_urls():
    print(category)

http://cnn.com
http://cnn.com/regions
http://cnn.com/opinions
http://cnn.com/asia
http://cnn.com/weather
http://cnn.com/collection
http://cnn.com/transcripts
http://cnn.com/election
http://cnn.com/sport
http://cnn.com/us
http://cnn.com/china
http://cnn.com/specials
http://money.cnn.com
http://cnn.com/more
http://commercial.cnn.com
http://cnn.com/politics
http://cnn.com/africa
http://cnn.com/msa
http://cnnpressroom.blogs.cnn.com
http://cnn.com/travel
http://cnn.com/europe
http://cnn.com/health
http://cnn.com/style
http://cnn.com/americas
http://cnn.com/middle-east
http://cnn.com/vr
http://go.cnn.com
http://cnn.com/entertainment


In [46]:
article = cnn_paper.articles[0]

In [47]:
article.download()

In [48]:
article.html

'<!DOCTYPE html><html class="no-js"><head><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"><meta charset="utf-8"><meta content="text/html" http-equiv="Content-Type"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0"><script type="text/javascript" async="false">(function(win, doc) {var bannerElement;win.enableSmartBannerTracking = false;if (typeof navigator.userAgent === \'string\' && navigator.userAgent.search(/(iPhone|iPad|iPod);/) !== -1 &&typeof doc.cookie === \'string\' && doc.cookie.search(/(^|;)\\s*CNNtosAgreed=[^;]+/) !== -1) {bannerElement = doc.createElement(\'meta\');win.enableSmartBannerTracking = true;bannerElement.name = "apple-itunes-app";bannerElement.content = "app-id=331786748";doc.getElementsByTagName(\'head\')[0].appendChild(bannerElement);}}(window, document));</script><link href="/favicon.ie9.ico" rel="Shortcut Icon" type="image/x-icon"/><link href="//cdn.cnn.com/cnn/.e/img/3.0/global/misc/apple-touch-icon.png" rel

In [49]:
article.parse()

In [50]:
article.authors

[]

In [51]:

article.top_image

'https://cdn.cnn.com/cnnnext/dam/assets/170304155524-vr-how-to-watch-cover-image-super-tease.jpg'